### This notebook is for testing model recommendation with the prepared testing dataset
### The model imported is expected to be on hybrid level with column names as follows
### ['origin', 'destination', 'ship method', 'wh_ind', 'pad'] where the origin can be either warehouse id (wh_ind =1) or origin zip3 (wh_ind = 0), destination can be destination zip3 for 3p shipments and hard-coded 'SWA_DEST' for 1p shipments, 'pad' is the model output


In [11]:
import pandas as pd
import numpy as np
import timeit
import matplotlib.pyplot as plt
import glob
import collections
import os

In [2]:
def schedule_adjust(test_data_input):
    
    test_data = test_data_input.copy()
    test_data.loc[test_data['pad'] == 0, 'adjusted_pad'] = 0
    
    ### positive and negative pad
    ## mon-fri delivery
    test_data.loc[(test_data['pad'] >0) 
              & (test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'].isnull())
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([0,1,2,3,4])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] <=4), 'adjusted_pad'] \
    = test_data.loc[(test_data['pad'] >0) 
              & (test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'].isnull())
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([0,1,2,3,4])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] <=4), 'pad']
    
    test_data.loc[(test_data['pad'] >0) 
              & (test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'].isnull())
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([0,1,2,3,4])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] >4), 'adjusted_pad'] \
    = test_data.loc[(test_data['pad'] >0) 
              & (test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'].isnull())
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([0,1,2,3,4])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] >4), 'pad'] + 2
    test_data.loc[
              (test_data['pad'] >0) 
              &(test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'].isnull())
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([5])) , 'adjusted_pad'] \
    = test_data.loc[
                (test_data['pad'] >0) 
              &(test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'].isnull())
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([5])) , 'pad'] + 1
    test_data.loc[(test_data['pad'] >0) 
              & (test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'].isnull())
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([6])) , 'adjusted_pad'] \
    = test_data.loc[(test_data['pad'] >0) 
              & (test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'].isnull())
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([6])) , 'pad'] 
    # negative pads
    test_data.loc[(test_data['pad'] <0) 
              & (test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'].isnull())
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([0,1,2,3,4])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] >= 0), 'adjusted_pad'] \
    = test_data.loc[(test_data['pad'] <0) 
              & (test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'].isnull())
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([0,1,2,3,4])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] >= 0), 'pad']
    
    test_data.loc[(test_data['pad'] <0) 
              & (test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'].isnull())
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([0,1,2,3,4])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] < 0), 'adjusted_pad'] \
    = test_data.loc[(test_data['pad'] <0) 
              & (test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'].isnull())
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([0,1,2,3,4])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] < 0), 'pad'] - 2
    
    test_data.loc[
              (test_data['pad'] <0) 
              &(test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'].isnull())
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([5])) , 'adjusted_pad'] \
    = test_data.loc[
                (test_data['pad'] <0) 
              &(test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'].isnull())
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([5])) , 'pad'] 
    test_data.loc[(test_data['pad'] <0) 
              & (test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'].isnull())
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([6])) , 'adjusted_pad'] \
    = test_data.loc[(test_data['pad'] <0) 
              & (test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'].isnull())
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([6])) , 'pad'] - 1
    
    
    
    # mon-sun delivery
    test_data.loc[(test_data['pad'].notnull())        
              & (test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'] == 'Y'), 'adjusted_pad'] \
    = test_data.loc[(test_data['pad'].notnull())        
              & (test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'] == 'Y'), 'pad'] 
    
        # mon-sat delivery
    test_data.loc[(test_data['pad'] >0) 
              & (test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([0,1,2,3,4,5])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] >5), 'adjusted_pad'] \
    = test_data.loc[(test_data['pad'] >0) 
              &(test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([0,1,2,3,4,5])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] >5), 'pad'] + 1
    
    test_data.loc[
              (test_data['pad'] >0) 
              &(test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([0,1,2,3,4,5])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] <=5), 'adjusted_pad'] \
    = test_data.loc[(test_data['pad'] >0) 
              & (test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([0,1,2,3,4,5])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] <=5), 'pad'] 
    
    test_data.loc[(test_data['pad'] >0) 
              & (test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([6])), 'adjusted_pad'] \
    = test_data.loc[(test_data['pad'] >0) 
              & (test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([6])), 'pad'] 
    
    # negative pads
    test_data.loc[(test_data['pad'] <0) 
              & (test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([0,1,2,3,4,5])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] >=0), 'adjusted_pad'] \
    = test_data.loc[(test_data['pad'] <0) 
              &(test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([0,1,2,3,4,5])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] >=0), 'pad'] 
    
    test_data.loc[
              (test_data['pad'] <0) 
              &(test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([0,1,2,3,4,5])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] <0), 'adjusted_pad'] \
    = test_data.loc[(test_data['pad'] <0) 
              & (test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([0,1,2,3,4,5])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] <0), 'pad'] - 1
    
    test_data.loc[(test_data['pad'] <0) 
              & (test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([6])), 'adjusted_pad'] \
    = test_data.loc[(test_data['pad'] <0) 
              & (test_data['mon'] == 'Y') 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([6])), 'pad'] 
    
    # tu-sat delivery
    
    test_data.loc[(test_data['pad'] >0) 
              &(test_data['mon'].isnull()) 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([1,2,3,4,5])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] >5), 'adjusted_pad'] \
    = test_data.loc[
              (test_data['pad'] >0) 
              &(test_data['mon'].isnull()) 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([1,2,3,4,5])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] >5), 'pad'] + 2
    
    test_data.loc[(test_data['pad'] >0) 
              & (test_data['mon'].isnull()) 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([1,2,3,4,5])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] <=5), 'adjusted_pad'] \
    = test_data.loc[(test_data['pad'] >0) 
              & (test_data['mon'].isnull()) 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([1,2,3,4,5])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] <=5), 'pad'] 
    
    test_data.loc[(test_data['pad'] >0) 
              & (test_data['mon'].isnull()) 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([6])) , 'adjusted_pad'] \
    = test_data.loc[(test_data['pad'] >0) 
              &(test_data['mon'].isnull()) 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([6])), 'pad'] + 1
    
    test_data.loc[(test_data['pad'] >0) 
              & (test_data['mon'].isnull()) 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([0])) , 'adjusted_pad'] \
    = test_data.loc[(test_data['pad'] >0) 
              & (test_data['mon'].isnull()) 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([0])), 'pad'] 
    # negative pads
    test_data.loc[(test_data['pad'] <0) 
              &(test_data['mon'].isnull()) 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([1,2,3,4,5])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] >=1), 'adjusted_pad'] \
    = test_data.loc[
              (test_data['pad'] <0) 
              &(test_data['mon'].isnull()) 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([1,2,3,4,5])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] >=1), 'pad'] 
    
    test_data.loc[(test_data['pad'] <0) 
              & (test_data['mon'].isnull()) 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([1,2,3,4,5])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] <1), 'adjusted_pad'] \
    = test_data.loc[(test_data['pad'] <0) 
              & (test_data['mon'].isnull()) 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([1,2,3,4,5])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] <1), 'pad'] - 2
    
    test_data.loc[(test_data['pad'] <0) 
              & (test_data['mon'].isnull()) 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([6])) , 'adjusted_pad'] \
    = test_data.loc[(test_data['pad'] <0) 
              &(test_data['mon'].isnull()) 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([6])), 'pad'] 
    
    test_data.loc[(test_data['pad'] <0) 
              & (test_data['mon'].isnull()) 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([0])) , 'adjusted_pad'] \
    = test_data.loc[(test_data['pad'] <0) 
              & (test_data['mon'].isnull()) 
              & (test_data['tu'] == 'Y') 
              & (test_data['wed'] == 'Y')
              & (test_data['th'] == 'Y')
              & (test_data['fri'] == 'Y')
              & (test_data['sat'] == 'Y')
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([0])), 'pad'] - 1
    
    
    # sat delivery only
    test_data.loc[(test_data['pad'] >0) 
              & (test_data['mon'].isnull()) 
              & (test_data['tu'].isnull()) 
              & (test_data['wed'].isnull())
              & (test_data['th'].isnull())
              & (test_data['fri'].isnull())
              & (test_data['sat'].isnull())
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([0,1,2,3,4,5])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] <=5), 'adjusted_pad']\
    = 5 - test_data.loc[(test_data['pad'] >0) 
              & (test_data['mon'].isnull()) 
              & (test_data['tu'].isnull()) 
              & (test_data['wed'].isnull())
              & (test_data['th'].isnull())
              & (test_data['fri'].isnull())
              & (test_data['sat'].isnull())
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([0,1,2,3,4,5])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] <=5), 'unpadded_pdd_dayofweek']
    
    test_data.loc[(test_data['pad'] >0) 
              & (test_data['mon'].isnull()) 
              & (test_data['tu'].isnull()) 
              & (test_data['wed'].isnull())
              & (test_data['th'].isnull())
              & (test_data['fri'].isnull())
              & (test_data['sat'].isnull())
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([0,1,2,3,4,5])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] >5), 'adjusted_pad'] \
    = 12 - test_data.loc[(test_data['pad'] >0) 
              & (test_data['mon'].isnull()) 
              & (test_data['tu'].isnull()) 
              & (test_data['wed'].isnull())
              & (test_data['th'].isnull())
              & (test_data['fri'].isnull())
              & (test_data['sat'].isnull())
              & (test_data['sun'].isnull())
              & (test_data['unpadded_pdd_dayofweek'].isin([0,1,2,3,4,5])) 
              & (test_data['unpadded_pdd_dayofweek'] + test_data['pad'] >5), 'unpadded_pdd_dayofweek']    
    return test_data


In [3]:
## Testing 
def ReadTestData(date):
     
    data = pd.read_csv('s3://df-datascience-adhoc/Adhoc/data/xinxiong/test/tt_pad_test_shipments_{}.csv000'.format(date))
    data.columns =['region_id', 'marketplace_id', 'shipment_id', 'tracking_id',
                   'warehouse_id', 'carrier', 'ship_method_orig', 'ship_method', 'origin_zip3', 'destination_zip3', 
                   'origin_zip5', 'destination_zip5',
                   'order_datetime','of_datetime', 'is_fasttrack', 
                   'c2exsd', 'c2p_days', 'c2p_days_unpadded', 'c2d_days', 'att_c2d_days',
                   'att_failed_pdd', 'no_att_scan', 'unpadded_pdd_date', 'tt_pad', 'gl_group',
                   'att_del_datetime', 'c2p_hours_unpadded', 'c2p_hours', 'c2d_hours',
                   'mon', 'tu', 'wed', 'th', 'fri', 'sat', 'sun', 'gl_label', 'amazon_parent_ship_method',
                   ]
    data['warehouse_id'] = data['warehouse_id'].astype('str')
    data['origin_zip3'] = data['origin_zip3'].fillna(0).astype('int').astype('str').str.zfill(3) 
    data['destination_zip3'] = data['destination_zip3'].fillna(0).astype('int').astype('str').str.zfill(3) 
    
    #data['origin_zip5'] = data['origin_zip5'].fillna(0).astype('int').astype('str').str.zfill(5) 
    #data['destination_zip5'] = data['destination_zip5'].fillna(0).astype('int').astype('str').str.zfill(5)
    
    data = data[(data['c2p_hours']>=0) & (data['c2p_days']>=0)].copy()
    ship_method_list  = ['UPS_GROUND','SWA','UPS_2ND_DAY','UPS_NEXT_DAY']
    data = data[data['ship_method'].isin(ship_method_list)]

    print("Test data shape is: ", data.shape)
    
    return data

In [4]:
 def apply_pads(test_data, model_output):
    pad_dict = {}
    for idx, row in model_output.iterrows():
        if row['ship_method'] == "SWA":
            key = (row['origin'], row['ship_method'])
        else:
            key = (row['origin'], row['destination_zip3'], row['ship_method'])

        pad_dict[key] = row['pad']

    def get_pad(row):
        if row['ship_method'] == "SWA":
            key = (row['warehouse_id'], row['ship_method'])
            return pad_dict.get(key, None)
        else:
            key = (row['warehouse_id'], row['destination_zip3'], row['ship_method'])
            if key in pad_dict.keys():
                return pad_dict.get(key)
            else:
                key = (row['origin_zip3'], row['destination_zip3'], row['ship_method'])
                if key in pad_dict.keys():
                    return pad_dict.get(key)
                


    test_data['pad'] = test_data.apply(get_pad, axis=1)

    padded_count = test_data['pad'].notna().sum()

    total_entries = len(test_data)
    percentage_padded = padded_count / total_entries * 100
    print(f"Percentage of shipments padded in testing period: {percentage_padded:.2f}%")

    return test_data

In [5]:
def ModelPerformance(test_data, model_output): # read models and test on testing data

    test_data['shipping_tracking'] = test_data['shipment_id'].astype('str') + test_data['tracking_id'].astype('str')    
    test_data['unpadded_pdd_date'] = pd.to_datetime(test_data['unpadded_pdd_date'])
    test_data['unpadded_pdd_dayofweek'] = test_data['unpadded_pdd_date'].dt.dayofweek
    
    test_data_with_pads = apply_pads(test_data.copy(), model_output.copy())
    
    print('Generating summary metrics by ship method...')
    result_sm = metrics_ship_method(test_data_with_pads)
    return result_sm

In [6]:
def calculate_weighted_average(result):
    if result['Model Coverage'].dtype == object:
        result['Model Coverage'] = result['Model Coverage'].astype(float)

    # Check for other percentage columns that may include '%' characters
    percentage_cols = ['Actual DEA (Optimized vol.)', 'Actual DEA (Non-optimized vol.)', 'UPRM DEA (Optimized vol.)', 'UPRM C2P (Optimized vol.)']
    for col in percentage_cols:
        if result[col].dtype == object:
            if result[col].astype(str).str.contains('%').any():
                result[col] = result[col].str.rstrip('%').astype(float) / 100
            else:
                result[col] = result[col].astype(float)

    if result['Shipment Count'].dtype == object:
        result['Shipment Count'] = result['Shipment Count'].replace(',', '').astype(int)

    # Calculate weighted values for each category
    result['weighted_dea_opt'] = result['UPRM DEA (Optimized vol.)'] * result['Model Coverage'] * result['Shipment Count']
    result['weighted_c2p_opt'] = result['UPRM C2P (Optimized vol.)'] * result['Model Coverage'] * result['Shipment Count']
    result['weighted_dea_non_opt'] = result['Actual DEA (Non-optimized vol.)'] * (1 - result['Model Coverage']) * result['Shipment Count']
    result['weighted_c2p_non_opt'] = result['Actual C2P (Non-optimized vol.)'] * (1 - result['Model Coverage']) * result['Shipment Count']
   
    # Compute weighted averages for each row rather than aggregating first
    result['Estimated DEA (Optimized+Non-optimized)'] = (result['weighted_dea_opt'] + result['weighted_dea_non_opt']) / result['Shipment Count']
    result['Estimated C2P (Optimized+Non-optimized)'] = (result['weighted_c2p_opt'] + result['weighted_c2p_non_opt']) / result['Shipment Count']
   
    result = result.drop(columns=['weighted_dea_opt', 'weighted_c2p_opt', 'weighted_dea_non_opt', 'weighted_c2p_non_opt'])
    return result


def metrics_ship_method(test_data): 
    result = metrics(test_data)
    ship_method = ['Overall']

    
    for sm in test_data['ship_method'].unique():
        print(f"Getting results for {sm}")
        test_data_s = test_data[test_data['ship_method'] == sm].copy()        
        result_s = metrics(test_data_s)
        result = pd.concat([result, result_s], axis = 0, ignore_index=True)
        ship_method.append(sm)
            
    result.columns = ['Shipment Count', 'Model Coverage', 'Overall Actual C2P', 'Overall Actual Speed', 'Unpadded C2P (Optimized vol.)', 'Unpadded DEA (Optimized vol.)', 'Actual C2P (Optimized vol.)',
                      'Actual DEA (Optimized vol.)', 'C2D', 'UPRM C2P (Optimized vol.)', 'UPRM DEA (Optimized vol.)', 'Unpadded C2P (Non-optimized vol.)',
                      'Unpadded DEA (Non-optimized vol.)', 'Actual C2P (Non-optimized vol.)', 'Actual DEA (Non-optimized vol.)',
                      'Pad %', 'TT Pad %', 'UTT Pad %']
    result.index = ship_method
    
    ship_pct = test_data.groupby('ship_method').apply(lambda x: x.shape[0]/test_data.shape[0])
    ship_pct_f = test_data.groupby('ship_method').apply(lambda x: '{:.2%}'.format(x.shape[0]/test_data.shape[0]))
    result = pd.concat([result, ship_pct_f, ship_pct], axis = 1)
    result.columns = ['Shipment Count', 'Model Coverage', 'Overall Actual DEA', 'Overall Actual C2P', 'Unpadded C2P (Optimized vol.)', 'Unpadded DEA (Optimized vol.)', 'Actual C2P (Optimized vol.)',
                      'Actual DEA (Optimized vol.)', 'C2D', 'UPRM C2P (Optimized vol.)', 'UPRM DEA (Optimized vol.)', 'Unpadded C2P (Non-optimized vol.)',
                      'Unpadded DEA (Non-optimized vol.)', 'Actual C2P (Non-optimized vol.)', 'Actual DEA (Non-optimized vol.)',
                      'Pad %', 'TT Pad %', 'UTT Pad %', 'volume_share', 'volume_share_1']
    result = result.sort_values(by = 'volume_share_1', ascending = False)
    result = result.drop('volume_share_1', axis = 1)
    
    # Perform the weighted average calculations for net DEA and C2P
    result = calculate_weighted_average(result)    
    return result

def metrics(test_data):
    # get the overall speed and DEA
    no_att_scan_overall = test_data.loc[test_data['no_att_scan'] == 'Y', 'shipping_tracking'].nunique()
    shipments_overall = test_data['shipping_tracking'].nunique()
    overall_speed = round((test_data['c2p_hours']/24).mean(), 2)      
    att_failed_pdd_current_overall = test_data.loc[
        (test_data['no_att_scan'] == 'N') &
        (test_data['c2p_days'] < test_data['att_c2d_days']),  # Use c2p_days if it includes pad adjustments
        'shipping_tracking'
    ].nunique()
    overall_dea = (
        (shipments_overall - att_failed_pdd_current_overall - no_att_scan_overall) /
        (shipments_overall - no_att_scan_overall)
    ) if (shipments_overall - no_att_scan_overall) != 0 else None
    overall_dea_formatted = '{:.2%}'.format(overall_dea) if overall_dea else None
    
    print("test_data shape: ", test_data.shape, "len shipments_overall", shipments_overall, overall_dea_formatted)

    
    test_data_model = test_data[test_data['pad'].notnull()].copy()
    test_data_nomodel = test_data[test_data['pad'].isnull()].copy()
    print("test_data_nomodel shape is:" , test_data_nomodel.shape)
        
    model_coverage = round(test_data_model.shape[0]/test_data.shape[0], 2)
    
    shipment_cnt = test_data.shape[0]
    
    
    ## shipment not covered by model ##
    no_att_scan_nomodel = test_data_nomodel.loc[test_data_nomodel['no_att_scan'] == 'Y', 'shipping_tracking'].nunique()
    shipments_nomodel = test_data_nomodel['shipping_tracking'].nunique()
 
    unpadded_speed_nomodel = round((test_data_nomodel['c2p_hours_unpadded']/24).mean(),2)         
    att_failed_pdd_unpadded_nomodel = test_data_nomodel.loc[(test_data_nomodel['no_att_scan'] == 'N') & (test_data_nomodel['c2p_days_unpadded'] < test_data_nomodel['att_c2d_days']), 'shipping_tracking'].nunique()    
    unpadded_dea_nomodel = (shipments_nomodel- att_failed_pdd_unpadded_nomodel - no_att_scan_nomodel)/(shipments_nomodel-no_att_scan_nomodel) if shipments_nomodel-no_att_scan_nomodel !=0 else None    
    unpadded_dea_nomodel = '{:.2%}'.format(unpadded_dea_nomodel) if unpadded_dea_nomodel else None
    
    
    current_speed_nomodel = round((test_data_nomodel['c2p_hours']/24).mean(),2)      
    att_failed_pdd_current_nomodel = test_data_nomodel.loc[test_data_nomodel['att_failed_pdd'] == 'Y', 'shipping_tracking'].nunique()
    current_dea_nomodel = (shipments_nomodel- att_failed_pdd_current_nomodel - no_att_scan_nomodel)/(shipments_nomodel-no_att_scan_nomodel) if shipments_nomodel-no_att_scan_nomodel !=0 else None
    current_dea_nomodel = '{:.2%}'.format(current_dea_nomodel) if current_dea_nomodel else None

    
    
    if test_data_model.shape[0] >0:
        print("test_data_model shape is:" , test_data_model.shape)
        test_data_model = schedule_adjust(test_data_model) # adjust pad for working day schedule
    
        no_att_scan = test_data_model.loc[test_data_model['no_att_scan'] == 'Y', 'shipping_tracking'].nunique()
        shipments = test_data_model['shipping_tracking'].nunique()
 
        unpadded_speed = round((test_data_model['c2p_hours_unpadded']/24).mean(),2)
        c2d = round((test_data_model['c2d_hours']/24).mean(),2)
    
    
        att_failed_pdd_unpadded = test_data_model.loc[(test_data_model['no_att_scan'] == 'N') & (test_data_model['c2p_days_unpadded'] < test_data_model['att_c2d_days']), 'shipping_tracking'].nunique()
    
        unpadded_dea = (shipments- att_failed_pdd_unpadded - no_att_scan)/(shipments-no_att_scan) if shipments-no_att_scan !=0 else None    

        current_speed = round((test_data_model['c2p_hours']/24).mean(),2)   

   
        att_failed_pdd_current = test_data_model.loc[test_data_model['att_failed_pdd'] == 'Y', 'shipping_tracking'].nunique()
        current_dea = (shipments- att_failed_pdd_current - no_att_scan)/(shipments-no_att_scan) if shipments-no_att_scan !=0 else None
    
        current_efficiency =  round((current_dea - unpadded_dea)*100/(current_speed - unpadded_speed), 2) if current_speed and unpadded_speed and current_dea and unpadded_dea and current_speed - unpadded_speed !=0 else None
        #current_efficiency_2 =  round(((current_dea - unpadded_dea)/unpadded_dea)/((current_speed - unpadded_speed)/unpadded_speed), 2) if current_speed - unpadded_speed !=0 else None
    
        current_dea = '{:.2%}'.format(current_dea) if current_dea else None

        new_speed = round(((test_data_model['c2p_hours_unpadded'] + test_data_model['adjusted_pad'] * 24)/24).mean(),2)
        att_failed_pdd_new = test_data_model.loc[(test_data_model['no_att_scan'] == 'N') & (test_data_model['c2p_days_unpadded'] + test_data_model['adjusted_pad'] < test_data_model['att_c2d_days']), 'shipping_tracking'].nunique()
        new_dea = (shipments- att_failed_pdd_new - no_att_scan)/(shipments-no_att_scan) if (shipments-no_att_scan) !=0 else None
        new_efficiency = round((new_dea - unpadded_dea)*100/(new_speed - unpadded_speed), 2) if new_speed and unpadded_speed and new_dea and unpadded_dea and new_speed - unpadded_speed !=0 else None
        #new_efficiency_2 =  round(((new_dea - unpadded_dea)/unpadded_dea)/((new_speed - unpadded_speed)/unpadded_speed), 2) if new_speed - unpadded_speed !=0 and new_dea and unpadded_dea else None

    
        new_dea = '{:.2%}'.format(new_dea) if new_dea else None
    
        unpadded_dea = '{:.2%}'.format(unpadded_dea) if unpadded_dea else None
    
        padded_pct = '{:.2%}'.format(test_data_model[test_data_model['adjusted_pad'] !=0].shape[0]/test_data_model.shape[0] if test_data_model.shape[0] >0 else None)
    
        padded_pct_noswa = '{:.2%}'.format(test_data_model[(test_data_model['adjusted_pad'] !=0) & (test_data_model['ship_method'] != 'SWA')].shape[0]/test_data_model.shape[0] if test_data_model.shape[0]>0 else None)
        padded_pct_swa = '{:.2%}'.format(test_data_model[(test_data_model['adjusted_pad'] !=0) & (test_data_model['ship_method'] == 'SWA')].shape[0]/test_data_model.shape[0] if test_data_model.shape[0] >0 else None)  
    
        # speed and dea by ups and swa    
        test_ups = test_data_model[test_data_model['ship_method'].isin(['UPS_GROUND', 'UPS_NEXT_DAY', 'UPS_2ND_DAY', 'UPS_3_DAY'])].copy()
        test_swa = test_data_model[test_data_model['ship_method'].isin(['SWA'])].copy()
    
        shipments_ups = test_ups['shipping_tracking'].nunique()
        new_speed_ups = round(((test_ups['c2p_hours_unpadded'] + test_ups['adjusted_pad'] * 24)/24).mean(),2)
        att_failed_pdd_new_ups = test_ups.loc[(test_ups['no_att_scan'] == 'N') & (test_ups['c2p_days_unpadded'] + test_ups['adjusted_pad'] < test_ups['att_c2d_days']), 'shipping_tracking'].nunique()
        no_att_scan_ups = test_ups.loc[test_ups['no_att_scan'] == 'Y', 'shipping_tracking'].nunique()
        new_dea_ups = (shipments_ups- att_failed_pdd_new_ups - no_att_scan_ups)/(shipments_ups-no_att_scan_ups) if (shipments_ups-no_att_scan_ups) !=0 else None   
        new_dea_ups = '{:.2%}'.format(new_dea_ups) if new_dea_ups else None    
    
        shipments_swa = test_swa['shipping_tracking'].nunique()
        new_speed_swa = round(((test_swa['c2p_hours_unpadded'] + test_swa['adjusted_pad'] * 24)/24).mean(),2)
        att_failed_pdd_new_swa = test_swa.loc[(test_swa['no_att_scan'] == 'N') & (test_swa['c2p_days_unpadded'] + test_swa['adjusted_pad'] < test_swa['att_c2d_days']), 'shipping_tracking'].nunique()
        no_att_scan_swa = test_swa.loc[test_swa['no_att_scan'] == 'Y', 'shipping_tracking'].nunique()
        new_dea_swa = (shipments_swa - att_failed_pdd_new_swa - no_att_scan_swa)/(shipments_swa-no_att_scan_swa) if (shipments_swa-no_att_scan_swa) !=0 else None
        new_dea_swa = '{:.2%}'.format(new_dea_swa) if new_dea_swa else None
    

        result = pd.DataFrame([shipment_cnt, model_coverage, overall_speed, overall_dea_formatted, unpadded_speed, unpadded_dea, current_speed, current_dea, c2d, new_speed, new_dea, unpadded_speed_nomodel, unpadded_dea_nomodel, current_speed_nomodel, current_dea_nomodel, padded_pct, padded_pct_noswa, padded_pct_swa]).T
    else:
        result = pd.DataFrame([None] * 16 + [shipment_cnt, model_coverage, unpadded_speed_nomodel, unpadded_dea_nomodel, current_speed_nomodel, current_dea_nomodel]).T
    
    return result

In [7]:
date = '20240301'
test_data_0301 = ReadTestData(date)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:273: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


Test data shape is:  (2013387, 38)


In [12]:
path = "./Feb_analysis/"
file_names = [
    "NEW_MODEL_feb_93per_88perc1P_AIRDEA0.94_AMXLExcluded.csv",
    "NEW_MODEL_feb_93per_90perc1P_AIRDEA0.9_AMXLExcluded.csv",
    "NEW_MODEL_feb_95per_86perc1P_AIRDEA0.88_AMXLExcluded.csv",
    "NEW_MODEL_feb_95per_86perc1P_AIRDEA0.93_AMXLExcluded.csv",
    "NEW_MODEL_feb_95per_86perc1P_AIRDEA0.9_AMXLExcluded.csv",
    "NEW_MODEL_feb_95per_88perc1P_AIRDEA0.9_AMXLExcluded.csv",
    "NEW_MODEL_feb_95per_90perc1P_AIRDEA0.90_AMXLExcluded.csv",
    "NEW_MODEL_feb_96per_86perc1P_AIRDEA0.93_AMXLExcluded.csv"
]

def ReadSolverModel(file_name):
    model = pd.read_csv(f"{path}{file_name}")
    model['origin'] = model['Warehouse']
    model['destination_zip3'] = model['Destination ZIP3'].astype(str)
    model['ship_method'] = model['Ship Method']
    model.rename(columns={'Pad': 'pad'}, inplace=True)
    model = model[['wh_ind', 'origin', 'destination_zip3', 'ship_method', 'pad']]
    # Overwrite negative padding from the model with 0 days
    print(f"Padding distribution before change for {file_name}:")
    print(model['pad'].value_counts())
    model['pad'] = model['pad'].apply(lambda x: 0 if x < 0 else x)
    print(f"Padding distribution after change for {file_name}:")
    print(model['pad'].value_counts())
    return model

for file_name in file_names:
    model_data = ReadSolverModel(file_name)
    result = ModelPerformance(test_data_0301, model_data)  # Ensure test_data_0301 is defined or passed correctly
   
    result_output_path = os.path.join(path, f"result_{file_name}")
    result.to_csv(result_output_path, index=False)
    print(f"Results saved to {result_output_path}")

/tmp/ipykernel_64022/3702636474.py:14: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  model = pd.read_csv(f"{path}{file_name}")


Padding distribution before change for NEW_MODEL_feb_93per_88perc1P_AIRDEA0.94_AMXLExcluded.csv:
pad
 0.0    304391
 1.0     71923
-1.0     32257
 2.0      8856
-2.0      4950
-3.0      1301
 3.0      1207
 4.0       153
 5.0        38
-1.5         1
-0.5         1
Name: count, dtype: int64
Padding distribution after change for NEW_MODEL_feb_93per_88perc1P_AIRDEA0.94_AMXLExcluded.csv:
pad
0.0    342901
1.0     71923
2.0      8856
3.0      1207
4.0       153
5.0        38
Name: count, dtype: int64
Percentage of shipments padded in testing period: 82.89%
Generating summary metrics by ship method...
test_data shape:  (2013387, 41) len shipments_overall 2012208 93.50%
test_data_nomodel shape is: (344560, 41)
test_data_model shape is: (1668827, 41)
Getting results for SWA
test_data shape:  (409977, 41) len shipments_overall 409977 89.24%
test_data_nomodel shape is: (295, 41)
test_data_model shape is: (409682, 41)
Getting results for UPS_GROUND
test_data shape:  (1288253, 41) len shipments_o

/tmp/ipykernel_64022/2488935193.py:49: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ship_pct = test_data.groupby('ship_method').apply(lambda x: x.shape[0]/test_data.shape[0])
/tmp/ipykernel_64022/2488935193.py:50: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ship_pct_f = test_data.groupby('ship_method').apply(lambda x: '{:.2%}'.format(x.shape[0]/test_data.shape[0]))


Results saved to ./Feb_analysis/result_NEW_MODEL_feb_93per_88perc1P_AIRDEA0.94_AMXLExcluded.csv


/tmp/ipykernel_64022/3702636474.py:14: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  model = pd.read_csv(f"{path}{file_name}")


...

Results saved to ./Feb_analysis/result_NEW_MODEL_feb_95per_90perc1P_AIRDEA0.90_AMXLExcluded.csv


/tmp/ipykernel_64022/3702636474.py:14: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  model = pd.read_csv(f"{path}{file_name}")


Padding distribution before change for NEW_MODEL_feb_96per_86perc1P_AIRDEA0.93_AMXLExcluded.csv:
pad
 0.0    259723
 1.0     96322
-1.0     28465
 2.0     24099
 3.0      7760
-2.0      4547
 4.0      2175
-3.0      1227
 5.0       758
-1.5         1
-0.5         1
Name: count, dtype: int64
Padding distribution after change for NEW_MODEL_feb_96per_86perc1P_AIRDEA0.93_AMXLExcluded.csv:
pad
0.0    293964
1.0     96322
2.0     24099
3.0      7760
4.0      2175
5.0       758
Name: count, dtype: int64
Percentage of shipments padded in testing period: 82.89%
Generating summary metrics by ship method...
test_data shape:  (2013387, 41) len shipments_overall 2012208 93.50%
test_data_nomodel shape is: (344560, 41)
test_data_model shape is: (1668827, 41)
Getting results for SWA
test_data shape:  (409977, 41) len shipments_overall 409977 89.24%
test_data_nomodel shape is: (295, 41)
test_data_model shape is: (409682, 41)
Getting results for UPS_GROUND
test_data shape:  (1288253, 41) len shipments_o

/tmp/ipykernel_64022/2488935193.py:49: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ship_pct = test_data.groupby('ship_method').apply(lambda x: x.shape[0]/test_data.shape[0])
/tmp/ipykernel_64022/2488935193.py:50: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ship_pct_f = test_data.groupby('ship_method').apply(lambda x: '{:.2%}'.format(x.shape[0]/test_data.shape[0]))


Results saved to ./Feb_analysis/result_NEW_MODEL_feb_96per_86perc1P_AIRDEA0.93_AMXLExcluded.csv


In [13]:
result

,Shipment Count,Model Coverage,Overall Actual DEA,Overall Actual C2P,Unpadded C2P (Optimized vol.),Unpadded DEA (Optimized vol.),Actual C2P (Optimized vol.),Actual DEA (Optimized vol.),C2D,UPRM C2P (Optimized vol.),UPRM DEA (Optimized vol.),Unpadded C2P (Non-optimized vol.),Unpadded DEA (Non-optimized vol.),Actual C2P (Non-optimized vol.),Actual DEA (Non-optimized vol.),Pad %,TT Pad %,UTT Pad %,volume_share,Estimated DEA (Optimized+Non-optimized),Estimated C2P (Optimized+Non-optimized)
UPS_GROUND,1288253,0.83,7.49,95.02%,6.65,91.39%,7.36,0.9510,4.43,7.51,0.9590,7.41,90.12%,8.12,0.9460,41.02%,41.02%,0.00%,63.98%,0.956790,7.6137
SWA,409977,1.00,7.68,89.24%,7.41,86.68%,7.68,0.8925,5.93,7.60,0.8939,6.36,73.50%,6.36,0.7350,18.69%,0.00%,18.69%,20.36%,0.893900,7.6
UPS_2ND_DAY,251676,0.64,5.59,93.70%,5.35,91.14%,5.69,0.9422,3.94,5.58,0.9329,5.18,89.69%,5.42,0.9276,16.18%,16.18%,0.00%,12.50%,0.930992,5.5224
UPS_NEXT_DAY,63481,0.47,4.25,89.68%,4.07,88.64%,4.33,0.9174,2.73,4.34,0.9156,3.97,85.43%,4.18,0.8784,18.09%,18.09%,0.00%,3.15%,0.895884,4.2552
Overall,2013387,0.83,7.19,93.50%,6.67,90.15%,7.23,0.9351,4.72,7.29,0.9396,6.48,89.53%,7.01,0.9343,32.74%,28.16%,4.59%,NaN,0.938699,7.2424
